In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os

In [2]:
raw_df = pd.DataFrame()
print(os.getcwd())
pickle_dir = os.getcwd()+ "\\Benchmarking\\test_merged_sample.pickle"
print(pickle_dir)
raw_df = pd.read_pickle(pickle_dir)#test_merged_sample.pickle")
print("Initial pickle contains", len(raw_df.index), "records")

C:\Users\Shivalik\Desktop\CMU\M19\Capstone
C:\Users\Shivalik\Desktop\CMU\M19\Capstone\Benchmarking\test_merged_sample.pickle
Initial pickle contains 914372 records


# Data Preprocessing for Benchmarking

## Count the number of unique business_sids in the input dataframe

In [3]:
%%time
business_sid_list = raw_df.iloc[:,1].unique()
total_customers_in_data = len(business_sid_list)
print("There are" , total_customers_in_data , "unique customers in the dataset")

There are 2811 unique customers in the dataset
Wall time: 131 ms


## Extract relevant columns for benchmarking

In [4]:
benchmarking_df = raw_df[['business_sid', 'weight', 'zone']]
benchmarking_df.head()

,business_sid,weight,zone
1232555,SOJW8FQCGI,2.0,6
1423326,XKVL2MMJL2,4.0,6
1516794,BE922CE458,10.0,8
1738984,BE922CE458,20.0,2
164310,VAQ6Q6TLDY,15.0,8


## Create pivot tables for volume and weight, and their respective zone-wise proportions

In [5]:
def create_volumeCount_pivotTable(arg_df):
    tempvoldf = arg_df.groupby(['business_sid', 'zone']).count()
    tempvoldf.columns = ['Count']
    volumeCount_pivotTable = pd.pivot_table(tempvoldf, values = 'Count', index = ['business_sid'], columns = 'zone', 
                     aggfunc = np.sum, fill_value = 0)
    if len(volumeCount_pivotTable)==total_customers_in_data:
        return volumeCount_pivotTable
    else:
        return "There is an error in your code"
    
def create_volumeProportion_pivotTable(arg_df):
    volumeProportion_pivotTable = arg_df.apply(lambda x: x/x.sum(), axis = 1)
    if len(volumeProportion_pivotTable)==total_customers_in_data:
        return volumeProportion_pivotTable
    else:
        return "There is an error in your code"

    
def create_weightCount_pivotTable(arg_df):
    tempwtdf = arg_df.groupby(['business_sid', 'zone']).sum()
    weightSum_pivotTable = pd.pivot_table(tempwtdf, values = 'weight', index = ['business_sid'], columns = 'zone',
                                         aggfunc = np.sum, fill_value = 0)
    if len(weightSum_pivotTable)==total_customers_in_data:
        return weightSum_pivotTable
    else:
        return "There is an error in your code"
    

def create_weightProportion_pivotTable(arg_df):
    weightProportion_pivotTable= arg_df.apply(lambda x: x/x.sum(), axis = 1)
    if len(weightProportion_pivotTable)==total_customers_in_data:
        return weightProportion_pivotTable
    else:
        return "There is an error in your code"


In [6]:
def create_PivotTables(arg_df):
    a = create_volumeCount_pivotTable(arg_df)
    #print(a.head())
    b = create_volumeProportion_pivotTable(a)
    #print(b.head())
    c = create_weightCount_pivotTable(arg_df)
    #print(c.head())
    d = create_weightProportion_pivotTable(c)
    #print(d.head())
    return a,b,c,d
    

In [7]:
vc_table_test = create_volumeCount_pivotTable(benchmarking_df)
vc_table_test.head()

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0,0,1,1,1,1,2,0,1,0
0071F0E134,0,0,2505,1556,1193,711,61,20,3,0
008936A589,0,0,28,9,8,14,9,1,18,0
009F7F037A,0,0,7,18,12,37,19,6,31,0
00B9362597,0,0,70,22,6,17,12,4,0,0


In [8]:
vp_table_test = create_volumeProportion_pivotTable(vc_table_test)
vp_table_test.head()

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0.0,0.0,0.142857,0.142857,0.142857,0.142857,0.285714,0.000000,0.142857,0.0
0071F0E134,0.0,0.0,0.414118,0.257233,0.197223,0.117540,0.010084,0.003306,0.000496,0.0
008936A589,0.0,0.0,0.321839,0.103448,0.091954,0.160920,0.103448,0.011494,0.206897,0.0
009F7F037A,0.0,0.0,0.053846,0.138462,0.092308,0.284615,0.146154,0.046154,0.238462,0.0
00B9362597,0.0,0.0,0.534351,0.167939,0.045802,0.129771,0.091603,0.030534,0.000000,0.0


In [9]:
wc_table_test = create_weightCount_pivotTable(benchmarking_df)
wc_table_test.head()

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0.0,0.0,3.00,4.0,7.0,24.0,17.00,0.0,2.0,0.0
0071F0E134,0.0,0.0,59686.60,31323.3,26509.7,16345.8,1834.00,464.0,24.0,0.0
008936A589,0.0,0.0,838.73,312.0,193.0,400.0,312.76,47.0,210.7,0.0
009F7F037A,0.0,0.0,127.00,384.0,222.0,717.0,307.00,79.0,726.0,0.0
00B9362597,0.0,0.0,1421.40,462.4,113.0,407.6,156.00,125.0,0.0,0.0


In [10]:
wp_table_test = create_weightProportion_pivotTable(wc_table_test)
wp_table_test.head()

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0.0,0.0,0.052632,0.070175,0.122807,0.421053,0.298246,0.000000,0.035088,0.0
0071F0E134,0.0,0.0,0.438268,0.230001,0.194656,0.120024,0.013467,0.003407,0.000176,0.0
008936A589,0.0,0.0,0.362429,0.134820,0.083399,0.172847,0.135149,0.020309,0.091047,0.0
009F7F037A,0.0,0.0,0.049571,0.149883,0.086651,0.279859,0.119828,0.030835,0.283372,0.0
00B9362597,0.0,0.0,0.529307,0.172190,0.042079,0.151784,0.058092,0.046548,0.000000,0.0


In [11]:
#These table outputs are just a sample for testing

vsitestingdf = vc_table_test.head()
vditestingdf = vp_table_test.head()

wsitestingdf = wc_table_test.head()
wditestingdf = wp_table_test.head()

In [12]:
vsitestingdf

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0,0,1,1,1,1,2,0,1,0
0071F0E134,0,0,2505,1556,1193,711,61,20,3,0
008936A589,0,0,28,9,8,14,9,1,18,0
009F7F037A,0,0,7,18,12,37,19,6,31,0
00B9362597,0,0,70,22,6,17,12,4,0,0


In [13]:
vditestingdf

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0.0,0.0,0.142857,0.142857,0.142857,0.142857,0.285714,0.000000,0.142857,0.0
0071F0E134,0.0,0.0,0.414118,0.257233,0.197223,0.117540,0.010084,0.003306,0.000496,0.0
008936A589,0.0,0.0,0.321839,0.103448,0.091954,0.160920,0.103448,0.011494,0.206897,0.0
009F7F037A,0.0,0.0,0.053846,0.138462,0.092308,0.284615,0.146154,0.046154,0.238462,0.0
00B9362597,0.0,0.0,0.534351,0.167939,0.045802,0.129771,0.091603,0.030534,0.000000,0.0


In [14]:
wsitestingdf

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0.0,0.0,3.00,4.0,7.0,24.0,17.00,0.0,2.0,0.0
0071F0E134,0.0,0.0,59686.60,31323.3,26509.7,16345.8,1834.00,464.0,24.0,0.0
008936A589,0.0,0.0,838.73,312.0,193.0,400.0,312.76,47.0,210.7,0.0
009F7F037A,0.0,0.0,127.00,384.0,222.0,717.0,307.00,79.0,726.0,0.0
00B9362597,0.0,0.0,1421.40,462.4,113.0,407.6,156.00,125.0,0.0,0.0


In [15]:
wditestingdf

zone,0,1,2,3,4,5,6,7,8,9
business_sid,,,,,,,,,,
004BB4E8C4,0.0,0.0,0.052632,0.070175,0.122807,0.421053,0.298246,0.000000,0.035088,0.0
0071F0E134,0.0,0.0,0.438268,0.230001,0.194656,0.120024,0.013467,0.003407,0.000176,0.0
008936A589,0.0,0.0,0.362429,0.134820,0.083399,0.172847,0.135149,0.020309,0.091047,0.0
009F7F037A,0.0,0.0,0.049571,0.149883,0.086651,0.279859,0.119828,0.030835,0.283372,0.0
00B9362597,0.0,0.0,0.529307,0.172190,0.042079,0.151784,0.058092,0.046548,0.000000,0.0


## Calculate individual indices

In [16]:
def calculate_VSI(sid_in_focus, vsiPivotTable):
    sid_list = vsiPivotTable.index.values
    sid_count = len(sid_list)
    vsi_list = np.empty(sid_count)
    for x in range(sid_count):
        if(sid_list[x]!=sid_in_focus):
            vsi_numerator = vsiPivotTable.loc[sid_in_focus].combine(vsiPivotTable.iloc[x], min, 0)
            vsi_denominator = vsiPivotTable.loc[sid_in_focus].combine(vsiPivotTable.iloc[x], max, 0)
            vsi_list[x] = vsi_numerator.sum()/vsi_denominator.sum()
        else:
            vsi_list[x] = -1          
    return vsi_list

def calculate_VDI(sid_in_focus, vdiPivotTable):
    sid_list = vdiPivotTable.index.values
    sid_count = len(sid_list)
    vdi_list = np.empty(sid_count)
    for x in range(sid_count):
        if(sid_list[x]!=sid_in_focus):
            vdi_numerator = vdiPivotTable.loc[sid_in_focus].combine(vdiPivotTable.iloc[x], min, 0)
            vdi_denominator = vdiPivotTable.loc[sid_in_focus].combine(vdiPivotTable.iloc[x], max, 0)
            vdi_list[x] = vdi_numerator.sum()/vdi_denominator.sum()
        else:
            vdi_list[x] = -1          
    return vdi_list

def calculate_WSI(sid_in_focus, wsiPivotTable):
    sid_list = wsiPivotTable.index.values
    sid_count = len(sid_list)
    wsi_list = np.empty(sid_count)
    for x in range(sid_count):
        if(sid_list[x]!=sid_in_focus):
            wsi_numerator = wsiPivotTable.loc[sid_in_focus].combine(wsiPivotTable.iloc[x], min, 0)
            wsi_denominator = wsiPivotTable.loc[sid_in_focus].combine(wsiPivotTable.iloc[x], max, 0)
            wsi_list[x] = wsi_numerator.sum()/wsi_denominator.sum()
        else:
            wsi_list[x] = -1          
    return wsi_list

def calculate_WDI(sid_in_focus, wdiPivotTable):
    sid_list = wdiPivotTable.index.values
    sid_count = len(sid_list)
    wdi_list = np.empty(sid_count)
    for x in range(sid_count):
        if(sid_list[x]!=sid_in_focus):
            wdi_numerator = wdiPivotTable.loc[sid_in_focus].combine(wdiPivotTable.iloc[x], min, 0)
            wdi_denominator = wdiPivotTable.loc[sid_in_focus].combine(wdiPivotTable.iloc[x], max, 0)
            wdi_list[x] = wdi_numerator.sum()/wdi_denominator.sum()
        else:
            wdi_list[x] = -1          
    return wdi_list

In [17]:
a = calculate_VSI(sid_in_focus='0071F0E134', vsiPivotTable= vsitestingdf)
a

array([ 0.00115722, -1.        ,  0.01187335,  0.0167846 ,  0.02165647])

In [18]:
b = calculate_VDI(sid_in_focus='0071F0E134', vdiPivotTable= vditestingdf)
b

array([ 0.3857054 , -1.        ,  0.48002126,  0.2626598 ,  0.61132987])

In [19]:
c = calculate_WSI(sid_in_focus='0071F0E134', wsiPivotTable= wsitestingdf)
c

array([ 4.18540922e-04, -1.00000000e+00,  1.56003963e-02,  1.35876116e-02,
        1.97184174e-02])

In [20]:
d = calculate_WDI(sid_in_focus='0071F0E134', wdiPivotTable= wditestingdf)
d

array([ 0.23402045, -1.        ,  0.55972472,  0.26837476,  0.65212253])

## Calculate Similarity Score

In [22]:
#weight_array is the parameter for adjusting weights in the Similarity Score
weight_array = np.array([0.25,0.25,0.25,0.25])

#allPivots is the parameter for adjusting # of business_sids for which Similarity Score is calculated
#To select more than 5 business_sids,modify the x.head() argument below
allPivots = [x.head() for x in create_PivotTables(benchmarking_df)]
    
def calculate_similarity_score(sid_in_focus, pivotTables = allPivots, weights = weight_array,): 
    
    vsiList = calculate_VSI(sid_in_focus, vsiPivotTable= pivotTables[0]) 
    vdiList = calculate_VDI(sid_in_focus, vdiPivotTable= pivotTables[1]) 
    wsiList = calculate_WSI(sid_in_focus, wsiPivotTable= pivotTables[2]) 
    wdiList = calculate_WDI(sid_in_focus, wdiPivotTable= pivotTables[3]) 
    
    ss_array = (weight_array[0]*vsiList + \
                weight_array[1]*vdiList + \
                weight_array[2]*wsiList + \
                weight_array[3]*wdiList)*100
 
    return np.round(ss_array,3)    

## Create Similarity Score Table

In [23]:
#This function only takes the list of all relevant sids as an input and generates the complete similarity score table
#Note: This table takes up a large amount of memory if all business_sids are provided in allPivots

def create_similarity_score_table(sid_list):
    similarity_score_table = pd.DataFrame(index = sid_list, columns = sid_list).fillna(-1)
    for x in sid_list:
        similarity_score_table.loc[x] = calculate_similarity_score(sid_in_focus = x, pivotTables = allPivots, 
                                                                   weights = weight_array)
    return similarity_score_table

In [24]:
%%time
allPivots = [x.head(10) for x in create_PivotTables(benchmarking_df)]
sid_list = allPivots[0].index.values
create_similarity_score_table(sid_list)

Wall time: 3.22 s


In [25]:
create_similarity_score_table(sid_list)

,004BB4E8C4,0071F0E134,008936A589,009F7F037A,00B9362597,00BE684C10,00D34BA35B,00E31A2581,00EF728145,010010C13F
004BB4E8C4,-100.000,15.533,26.383,27.654,16.964,7.486,20.284,22.660,27.683,40.621
0071F0E134,15.533,-100.000,26.680,14.035,32.621,13.549,6.658,46.625,36.635,12.581
008936A589,26.383,26.680,-100.000,50.177,55.202,10.647,37.294,27.244,34.896,28.128
009F7F037A,27.654,14.035,50.177,-100.000,32.133,1.549,52.746,25.963,19.596,34.338
00B9362597,16.964,32.621,55.202,32.133,-100.000,18.331,21.029,24.482,47.021,14.230
00BE684C10,7.486,13.549,10.647,1.549,18.331,-100.000,2.480,5.843,19.875,3.359
00D34BA35B,20.284,6.658,37.294,52.746,21.029,2.480,-100.000,15.804,13.671,30.902
00E31A2581,22.660,46.625,27.244,25.963,24.482,5.843,15.804,-100.000,22.856,21.848
00EF728145,27.683,36.635,34.896,19.596,47.021,19.875,13.671,22.856,-100.000,23.885
010010C13F,40.621,12.581,28.128,34.338,14.230,3.359,30.902,21.848,23.885,-100.000


In [ ]:
#Sample IDs from Jose:
'''
7B8E9E45F5
E0518749DD
924052DDF5
AACOENJW
FHVD7BDTDG
'''